In [1]:
#Imports Modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [2]:
# Types
abs_path = os.getcwd()
form_types = pd.read_csv(abs_path + "\\dataset\\types.csv")
form_types.drop(form_types.columns[[0]], axis=1, inplace=True)

# Metadata
abs_path = os.getcwd()
df = pd.read_csv(abs_path + "\dataset\metadata.csv")
df.drop(df.columns[[0, 1, 3]], axis=1, inplace=True)
df[['file_name','simplified_type']]

#Convert Metadata to Dict
metadata = dict()
for i, row in df.iterrows():
    metadata[str(row['file_name'])] = np.where(form_types == row['simplified_type'])[0][0]

In [10]:
directory = abs_path + "\\dataset\\forms\\"
MAX_SIZE = 500
BATCH_SIZE = 32

In [4]:
samples_data = []
samples_data_type = []

for k, sample in enumerate(os.listdir(directory)):
    sample_array = np.zeros((MAX_SIZE, MAX_SIZE, 3))
    
    img = cv2.imread(directory+sample)
    
    if img.shape[0] > MAX_SIZE:
        h = img.shape[0]
        w = img.shape[1]
        
        ratio = MAX_SIZE / h
        img = cv2.resize(
            img,
            (int(w*ratio), int(h*ratio)),
            interpolation=cv2.INTER_CUBIC,
            )

    if img.shape[1] > MAX_SIZE:
        h = img.shape[0]
        w = img.shape[1]
        
        ratio = MAX_SIZE / w
        img = cv2.resize(
            img,
            (int(w*ratio), int(h*ratio)),
            interpolation=cv2.INTER_CUBIC,
            )
#         print(h, w, img.shape[0], img.shape[1])
        
    for i, _ in enumerate(img):
        for j, _ in enumerate(img[i]):
            sample_array[i][j] = img[i][j]
    
    samples_data.append(sample_array)
    samples_data_type.append(metadata[sample])
    
print("Completed All")
    
# Convert Lists to Numpy Arrays
samples_data = np.array(samples_data)
samples_data_type = np.array(samples_data_type)

with open(f'samples_data_{MAX_SIZE}_sim.npy', 'wb') as f:
    np.save(f, samples_data)
with open(f'samples_data_type_{MAX_SIZE}_sim.npy', 'wb') as f:
    np.save(f, samples_data_type)

Completed All


In [5]:
# samples_data = np.load(f'samples_data_{MAX_SIZE}_sim.npy')
# samples_data_type = np.load(f'samples_data_type_{MAX_SIZE}_sim.npy')

In [6]:
# Split once to get the test and training set
X_train, X_test, y_train, y_test = train_test_split(samples_data, samples_data_type, test_size=0.10, random_state=123)

# Split twice to get the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=123)

y_train = to_categorical(y_train, len(form_types))
y_test = to_categorical(y_test, len(form_types))
y_val = to_categorical(y_val, len(form_types))

In [7]:
model = models.Sequential()
# First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(MAX_SIZE, MAX_SIZE, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Third convolutional block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Fourth convolutional block
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Fifth convolutional block
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Flatten the output and add fully connected layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

# Output layer for 100 classes
model.add(Dense(len(form_types), activation='softmax'))
# Model is compiled. Adam is used to have a automatic learning rate
# Sparse Categorical Crossentropy is used since there are multiple drum types, but only one drum type per drum
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
#model.summary()


In [8]:
# datagen = ImageDataGenerator(
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=2,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest'
# )

# image = np.expand_dims(X_train[0], 0)

# datagen.fit(image)

# for x, val in zip(datagen.flow(
#     image,
#     save_to_dir=abs_path + "\\dataset\\aug\\",
#     save_prefix='aug',
#     save_format='png'),range(100)) :
#     pass


# datagen = ImageDataGenerator(
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=2,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest'
# )

# train_generator = train_datagen.flow(
#     X_train,
#     y_train,
#     batch_size=BATCH_SIZE,
# )
# val_datagen = ImageDataGenerator()
# validation_generator = val_datagen.flow(
#     X_val,
#     y_val,
#     batch_size=BATCH_SIZE
# )

# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(X_train) // train_generator.batch_size,
#     epochs=200,
#     validation_data=validation_generator,
#     validation_steps=len(X_val) // validation_generator.batch_size,
# )

In [11]:
# Fits model
EPOCHS = 50
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data= (X_val, y_val))

Epoch 1/50
 6/22 [=======>......................] - ETA: 1:58 - loss: 2.6448 - accuracy: 0.4688

KeyboardInterrupt: 

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
accuracy_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_accuracy_values=history_dict['val_accuracy']
epochs=range(1,51)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))

ax1.plot(epochs,loss_values,'blue',label='Training Loss')
ax1.plot(epochs,val_loss_values,'red', label='Validation Loss')
ax1.set_title('Losses')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(epochs,accuracy_values,'blue', label='Training accuracy')
ax2.plot(epochs,val_accuracy_values,'red',label='Validation accuracy')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# # Saves model to be used by the prediction Jupyter Notebook
model.save(f'saved_model\model_20240701_{MAX_SIZE}res_{EPOCHS}ep_sim')

# Test Model

In [ ]:
# Test Data
# model = tf.keras.models.load_model(abs_path + "/saved_model/model_20240630_100res_50ep")

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

MIN_CONF = 0.75

num_correct = np.array([])
conf_correct = np.array([])
filtered_total = np.array([])

for i, data in enumerate(X_test):
    
    data = np.expand_dims(X_test[i], axis=0)
    prediction = model.predict(data, verbose = 0)
    type_num = np.argmax(prediction, axis=1)
    
    test_data = y_test[i]
    correct_index = np.where(test_data == 1)[0][0]
    
    conf = prediction[:, type_num][0]
    correct = (correct_index == type_num)
    
    conf_correct = np.append(conf_correct, conf)

    if correct:
        num_correct = np.append(num_correct, True)
        continue
    
    num_correct = np.append(num_correct, False)
    
print(f"Before Removing Low Conf Predictions the model accuracy is: {np.sum(num_correct == 1) / len(X_test)}")
    
for i in range(len(num_correct)):
    if conf_correct[i] > MIN_CONF:
        filtered_total = np.append(filtered_total, num_correct[i])
        
filtered_correct = np.sum(filtered_total == 1)

print(f"After Removing Predictions with Conf <{MIN_CONF} the model accuracy is: {filtered_correct / len(filtered_total)}")

# 500 76%
# 200 87 % sim is best
# 100 67%
# 100 38% ImageGen
print("Total Test Size", len(X_test))
print("Total Correct", np.sum(num_correct == 1))
print("Total Filtered By Conf", len(filtered_total))
print("Correct Filtered By Conf", filtered_correct)